In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [1]:
from multi_env import make_reversi_vec_env, SelfPlayEnv
import torch as th
from players import RandomPlayer
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
import numpy as np

In [2]:
board_shape = 8
n_envs = 10
env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=n_envs,
    env_kwargs={
        'board_shape': board_shape,
        'LocalPlayer': RandomPlayer
    }
)

# Modificación de librería para que haga argmax solo sobre las válidas

In [3]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=0,
)

In [4]:
model.predict(env.reset())

(array([20, 39, 18,  7, 30, 61, 54, 33, 61, 25]), None)

# Custom ActorCriticPolicy 

https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/policies.py

In [5]:
from boardgame2 import ReversiEnv

In [6]:
env_not_vect = ReversiEnv(board_shape)

In [7]:
# env_not_vect.get_valid((state, player))

In [8]:
def get_actions_mask(state):
    player = 1
    valid_actions = env_not_vect.get_valid((state, player))
    return valid_actions.reshape(-1)  


In [9]:
get_actions_mask(env.reset()[0][0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [11]:
class CustomActorCriticPolicy(ActorCriticPolicy):
    def __init__(
        self,
        *args, # Todos los argumentos posicionales de ActorCriticPolicy
        actions_mask_func=None, # El nuevo argumento
        **kwargs # Todos los argumentos opcionales de ActorCriticPolicy
    ):
        super(CustomActorCriticPolicy, self).__init__(
            *args,
            **kwargs
        )
        if actions_mask_func:
            self.get_actions_mask = actions_mask_func 
    
    def sample_masked_actions(self, obs, distribution, deterministic=False, return_distribution=False):
        # Dada las obs y distribuciones luego de evaluar la red neuronal, samplear solo las acciones válidas
        # Las obs se usan para que con self.get_actions_mask se obtengan las acciones válidas
        # las distribuciones son el resultado de evaluar la red neuronal y van a dar acciones no validas
        # Generar una nueva distribución (del lado de los logits preferentemente) donde las acciones no válidas
        # tengan probabildad nula de ser muestreadas
        # Luego se modifican abajo los métodos
        # _predict, forward y evaluate_actions
        # Si tiene el flag de return_distribution en true devuelve la distribución nueva
        # Caso contrario devuelve las acciones
        # Para tener en cuenta, obs tiene dimensión [batch_size, channels, H, W]
        # Recomendamos poner un print(obs.shape)
        # y correr:
        # obs = env.reset()
        # actions, _ = model.predict(obs)
        # Para sacarse las dudas
        def get_mask(obs):
            masks = np.zeros((len(obs), obs.shape[-1] * obs.shape[-2]))
            for i, board in enumerate(obs):
                board = board[0].cpu().numpy()
                masks[i] = 1 - self.get_actions_mask(board)
            return th.from_numpy(masks).to(self.device)
            
        masks = get_mask(obs)
        masks[masks == 1] = -1e6 #np.inf
        masked_logits = distribution.logits + masks
        if return_distribution:
            return th.distributions.Categorical(logits=masked_logits)
        if deterministic:
            return th.argmax(masked_logits, axis=1)
        return th.distributions.Categorical(logits=masked_logits).sample()

    def _predict(self, observation, deterministic=False):
        """
        Get the action according to the policy for a given observation.
        :param observation:
        :param deterministic: Whether to use stochastic or deterministic actions
        :return: Taken action according to the policy
        """
        latent_pi, _, latent_sde = self._get_latent(observation)
        distribution = self._get_action_dist_from_latent(latent_pi, latent_sde)
        
        if self.get_actions_mask:
            actions = self.sample_masked_actions(observation, distribution.distribution, deterministic=deterministic)
        else:
            actions = distribution.get_actions(deterministic=deterministic)
        
        return actions
    
    def forward(self, obs: th.Tensor, deterministic: bool = False):
        """
        Forward pass in all the networks (actor and critic)
        :param obs: Observation
        :param deterministic: Whether to sample or use deterministic actions
        :return: action, value and log probability of the action
        """
        latent_pi, latent_vf, latent_sde = self._get_latent(obs)
        # Evaluate the values for the given observations
        values = self.value_net(latent_vf)
        distribution = self._get_action_dist_from_latent(latent_pi, latent_sde=latent_sde)
        distrib = self.sample_masked_actions(obs, distribution.distribution, return_distribution=True)
        
        
        if self.get_actions_mask:
            actions = self.sample_masked_actions(obs, distribution.distribution, deterministic=deterministic)
        else:
            actions = distribution.get_actions(deterministic=deterministic)

        # log_prob = distribution.log_prob(actions)
        log_prob = distrib.log_prob(actions)
        return actions, values, log_prob
    
    def evaluate_actions(self, obs: th.Tensor, actions: th.Tensor):
        """
        Evaluate actions according to the current policy,
        given the observations.
        :param obs:
        :param actions:
        :return: estimated value, log likelihood of taking those actions
            and entropy of the action distribution.
        """
        latent_pi, latent_vf, latent_sde = self._get_latent(obs)
        distribution = self._get_action_dist_from_latent(latent_pi, latent_sde)
        distrib = self.sample_masked_actions(obs, distribution.distribution, return_distribution=True)

        log_prob = distrib.log_prob(actions)
        values = self.value_net(latent_vf)
        return values, log_prob, distrib.entropy()

In [12]:
model = PPO(
    CustomActorCriticPolicy,
    env,
    verbose=0,
    policy_kwargs = {'actions_mask_func': get_actions_mask}
)


In [13]:
# Testeo de predict
model.policy.get_actions_mask(env.reset()[0][0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [14]:
obs = env.reset()
actions, _ = model.predict(obs)

In [15]:
# Verificar que las acciones son válidas
actions

array([21, 34, 20, 19, 26, 37, 29, 34, 37, 26])

In [16]:
# Testeo de forward
model.policy(th.from_numpy(obs).to(model.device))

(tensor([19, 43, 29, 19, 44, 19, 29, 20, 21, 44]),
 tensor([[ 0.1738],
         [-0.1327],
         [-0.1327],
         [ 0.2366],
         [-0.5164],
         [ 0.2366],
         [-0.1327],
         [-0.1327],
         [ 0.2366],
         [-0.3896]], grad_fn=<AddmmBackward>),
 tensor([-1.0978, -1.3846, -1.3869, -1.0972, -1.1016, -1.0972, -1.3869, -1.3900,
         -1.1056, -1.0984], dtype=torch.float64, grad_fn=<SqueezeBackward1>))

# Corremos PPO

In [17]:
board_shape = 8
n_envs = 6
gamma = 0.99
ent_coef = 0.0
gae_lambda = 0.95
n_epochs = 10 # 10 en la anterior

In [18]:
prefix = 'Reversi_PPO'
suffix = 'masked_actions'
model_name = f'{prefix}_{board_shape}by{board_shape}_{gamma}_{gae_lambda}_{ent_coef}_{n_epochs}_{n_envs}_{suffix}'
best_model_save_path = f'./models/{model_name}'
print(model_name)
print(best_model_save_path)

Reversi_PPO_8by8_0.99_0.95_0.0_10_6_masked_actions
./models/Reversi_PPO_8by8_0.99_0.95_0.0_10_6_masked_actions


In [19]:
model = PPO(
    CustomActorCriticPolicy,
    env,
    verbose=0,
    tensorboard_log='tensorboard_log',
    gamma=gamma,
    gae_lambda=gae_lambda,
    ent_coef=ent_coef,
    n_epochs=n_epochs,
    policy_kwargs = {'actions_mask_func': get_actions_mask}
)

In [20]:
from stable_baselines3.common.callbacks import EvalCallback

In [21]:
# El entorno de evaluación no corre en paralelo por eso uno solo
eval_env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=1,
    env_kwargs={
        'board_shape': board_shape,
        'LocalPlayer': RandomPlayer
    }
)

In [22]:
eval_callback = EvalCallback(
    eval_env = eval_env,
    eval_freq=1_000,
    n_eval_episodes=500,
    deterministic=True,
    verbose=1,
    best_model_save_path=best_model_save_path,
) 

In [23]:
model.learn(total_timesteps=int(1e10), callback=[eval_callback])

Eval num_timesteps=10000, episode_reward=-0.05 +/- 0.97
Episode length: 29.95 +/- 0.64
New best mean reward!
Eval num_timesteps=20000, episode_reward=-0.00 +/- 0.98
Episode length: 29.97 +/- 0.56
New best mean reward!
Eval num_timesteps=30000, episode_reward=0.15 +/- 0.98
Episode length: 30.05 +/- 0.63
New best mean reward!
Eval num_timesteps=40000, episode_reward=0.12 +/- 0.97
Episode length: 30.03 +/- 0.54
Eval num_timesteps=50000, episode_reward=0.30 +/- 0.93
Episode length: 29.97 +/- 1.03
New best mean reward!
Eval num_timesteps=60000, episode_reward=0.40 +/- 0.90
Episode length: 30.05 +/- 0.61
New best mean reward!
Eval num_timesteps=70000, episode_reward=0.48 +/- 0.85
Episode length: 30.10 +/- 0.55
New best mean reward!
Eval num_timesteps=80000, episode_reward=0.38 +/- 0.91
Episode length: 30.05 +/- 0.56
Eval num_timesteps=90000, episode_reward=0.53 +/- 0.84
Episode length: 30.01 +/- 0.56
New best mean reward!
Eval num_timesteps=100000, episode_reward=0.57 +/- 0.81
Episode length